In [1]:
import numpy as np
import pandas as pd
import re

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
import scipy.stats as st

from sklearn.utils import resample
from sklearn.metrics import *
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns


import warnings
warnings.filterwarnings('ignore')

In [2]:
class Data:
    """
    Load the data categorize the columns to categorical and numerical
    """
    def __init__(self, filepath: str = None, targetcol: str = None):
        self.data = pd.read_csv(filepath)
        self.targetcol = targetcol

    def get_numericals(self):
        self.num_cols = list(self.data.dtypes[
            (self.data.dtypes == 'int64') | (self.data.dtypes == 'float64')| 
            (self.data.dtypes == 'int32') | (self.data.dtypes == 'float32')|
            (self.data.dtypes == 'int16') | (self.data.dtypes == 'float16')|
            (self.data.dtypes == 'int8')].keys())
        if self.targetcol in self.num_cols:
            self.num_cols.remove(self.targetcol)
        self.cat_cols = list(set(self.data.columns).difference(self.num_cols))
        if self.targetcol in self.cat_cols:
            self.cat_cols.remove(self.targetcol)
        return self.num_cols, self.cat_cols

    def get_inputs(self):
        num_cols, cat_cols = self.get_numericals()
        return num_cols, self.targetcol, self.data, cat_cols

In [3]:
def vif(targetcol, data, num_cols, cat_cols):
    """
    Calculate the VIF and get the columns having multicollinear features (high VIF).
    args:
        targetcol: target column name
        data: whole cleaned dataframe
        num_cols: List of numerical column names
        cat_cols: List of categorical column names.
    return:
        vif_high: list of features having high VIF>10
    """
    y, X = dmatrices(targetcol+'~'+'+'.join(num_cols+cat_cols), data=data, return_type='dataframe')
    vif = pd.DataFrame()
    vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif["features"] = X.columns
    vif_high = list(vif[vif['VIF Factor']>10]['features'])
    if 'Intercept' in vif_high:
        vif_high.remove('Intercept')
    vif_high = list(set([re.sub(r'\[.*', '', s) for s in vif_high]))
    return vif_high


def removal_on_vif(targetcol:str=None, data:pd.DataFrame=None, num_cols:list=None, cat_cols:list=None):
    """
    Remove all the features having high VIF.
    args:
        targetcol: target column name
        data: whole cleaned dataframe
        num_cols: List of numerical column names
        cat_cols: List of categorical column names
    return: 
        num_cols: list containing numerical column names after removal based on VIF
        cat_cols: list containing categorical column names after removal based on VIF
    """
    vif_first = vif(targetcol, data, num_cols, cat_cols)
    print("vif first:",vif_first)
    feat_removal=[]
    for i in vif_first:
        cols = list(set(num_cols).difference(vif_first))
        cols.append(i)
        cols1 = list(set(cat_cols).difference(vif_first))
        ans = vif(targetcol,data[cols+cols1+[targetcol]],cols,list(set(cat_cols).difference(vif_first)))
        for j in ans:
            if j in vif_first:
                feat_removal.append(j)
    feat_removal = list(set([re.sub(r'\[.*', '', s) for s in feat_removal]))
    num_cols = list(set(num_cols).difference(feat_removal))
    cat_cols = list(set(cat_cols).difference(feat_removal))
    print(f"Columns Removed on basis of VIF: {feat_removal}")
    return num_cols, cat_cols

In [4]:
def removal_on_pvalue(targetcol: str = None, num_cols: list = None, cat_cols: list = None, data:pd.DataFrame=None):
    """
        Fit model and remove the columns based on pvalue test on each feature.
    args:
        targetcol: target column name
        data: whole cleaned dataframe
        num_cols: List of numerical column names
        cat_cols: List of categorical column names
    """
    model = smf.ols(targetcol+'~'+'+'.join(num_cols+cat_cols),data=data).fit()
    pvalue_removal = list(model.pvalues[model.pvalues>0.05].keys())
    if 'Intercept' in pvalue_removal:
        pvalue_removal.remove("Intercept")
    pvalue_removal = list(set([re.sub(r'\[.*', '', s) for s in pvalue_removal]))
    num_cols = list(set(num_cols).difference(pvalue_removal))
    cat_cols = list(set(cat_cols).difference(pvalue_removal))
    print("Columns removed on basis of Pvalue:", pvalue_removal)
    return num_cols, cat_cols

In [5]:
def anovatyp1_removal(targetcol: str = None, num_cols: list = None, cat_cols: list = None, data:pd.DataFrame=None):
    """
        Fit anova test and remove the columns failing the anova type 1 test.
    args:
        targetcol: target column name
        data: whole cleaned dataframe
        num_cols: List of numerical column names
        cat_cols: List of categorical column names
        """
    model = smf.ols(targetcol+'~'+'+'.join(num_cols+cat_cols),data=data).fit()
    anova_typ1 = sm.stats.anova_lm(model, typ=1)
    m = anova_typ1['PR(>F)']
    anovatyp1_removal = list(m[m>0.05].index)
    print("columns removed on basis of anova typ1:",anovatyp1_removal)
    num_cols = list(set(num_cols).difference(anovatyp1_removal))
    cat_cols = list(set(cat_cols).difference(anovatyp1_removal))
    return num_cols, cat_cols

In [6]:
def finalmodel(targetcol: str = None, num_cols: list = None, cat_cols: list = None, data:pd.DataFrame=None):
    """
    Fit final OLS model on data and also get explainability based on tstat values.
    args:
        targetcol: target column name
        data: whole cleaned dataframe
        num_cols: List of numerical column names
        cat_cols: List of categorical column names
    """
    print(f"Final Model:{targetcol}+'~'+{'+'.join(num_cols+cat_cols)}")
    num_cols, cat_cols = removal_on_pvalue(targetcol, num_cols, cat_cols, data)
    model = smf.ols(targetcol+'~'+'+'.join(num_cols+cat_cols),data=data).fit()
    explanation = np.abs(model.tvalues).sort_values(ascending=False)
    sns.regplot(x = data[targetcol], y = model.fittedvalues)
    return model, explanation, num_cols, cat_cols

In [7]:
def adjusted_r2(predictions:list=None, data:str=pd.DataFrame, targetcol:str=None):
    """
    Calculate adjusted R2.
    args:
        predictions: list of all the fitted model prediction values.
        data: DataFrame of initial dataset containing all columns.
        targetcol: target column name.
    """
#     try:
    r2 = r2_score(predictions,data[targetcol])
    adjr2 = 1-(1-r2)*(len(data)-1)/(len(data)-len(data.columns)-1)
    return adjr2
#     except:
#         print("Unable to calculate adjusted R2, please check predictions list.")
    

def bootstrap_confidence(n_iterations, targetcol, num_cols, cat_cols, data):
    """
    get confidence score of the model using bootstrapping.
    args:
        targetcol: target column name
        data: whole cleaned dataframe
        num_cols: List of numerical column names
        cat_cols: List of categorical column names
        n_iterations: number of iteration to run bootstrapping
    return:
        Confidence interval: Lower and upper quantile.
    """
    try:
        stats = list()
        for i in range(n_iterations):
            # prepare train and test sets
            train, test, y_train, y_test = train_test_split(data[num_cols+cat_cols], data[targetcol], test_size=0.2)    
            train[targetcol] = y_train
            test[targetcol] = y_test
            # fit model
            model = smf.ols(targetcol+'~'+'+'.join(num_cols+cat_cols),data=train).fit()
            # evaluate model
            predictions = model.predict(test[num_cols+cat_cols])
            score = adjusted_r2(predictions,test, targetcol)
            if i%100 == 0:
                print(f"Adj R2 for iter {i}: {np.round(score,3)*100}%")
            stats.append(score)
        alpha = 0.95
        p = ((1.0-alpha)/2.0) * 100
        lower = max(0.0, np.percentile(stats, p))
        p = (alpha+((1.0-alpha)/2.0)) * 100
        upper = min(1.0, np.percentile(stats, p))
        print('%.1f%% confidence interval %.1f%% and %.1f%%'%(alpha*100, lower*100, upper*100))
        return lower*100, upper*100
    except:
        print("not enough resampled data")
        return 0,0

In [8]:
def ci_pi(targetcol, num_cols, cat_cols, data):
    """
    Calculate Confidence Interval and Prediction Interval for a given Sample
    args:
        targetcol: target column name.
        num_cols: list of numerical columns.
        cat_cols: list of categorical columns.
        data: DataFrame of whole initial dataset. 
    return:
        2 tuples Confidence Interval and Prediction Interval
    """
    y, X = dmatrices(targetcol+'~'+'+'.join(num_cols+cat_cols), data=data, return_type='dataframe')
    stderr = np.sqrt(mse*np.dot(X.iloc[0].values,np.dot(np.linalg.inv(np.dot(X.T,X)),X.iloc[0].values.T)))
    t_conf = st.t.ppf(0.025,len(X)-len(X.columns))
    y_hat = model.predict(pd.DataFrame(data[num_cols+cat_cols].iloc[0:3]))[0]
    conf_interval = y_hat-(np.abs(t_conf)*stderr), y_hat+(np.abs(t_conf)*stderr)
    pred_interval = y_hat-(np.abs(t_conf)*np.sqrt((stderr**2)+mse)), y_hat+(np.abs(t_conf)*np.sqrt((stderr**2)+mse))
    return conf_interval, pred_interval

In [9]:
#Still left
#influential points is left out
#Model Assumptions
#if anyone vif is high remove all the dummy column and origin base column

In [ ]:
if __name__=="__main__":
    data_obj = Data("cleaned_data_project1.csv","price")
    num_cols, targetcol, data, cat_cols = data_obj.get_inputs()
    data.info()
    model = smf.ols(targetcol+'~'+'+'.join(num_cols+cat_cols),data=data).fit()
    #print(model.summary())
    sns.regplot(data[targetcol],model.fittedvalues)
    num_cols, cat_cols = removal_on_vif(targetcol, data, num_cols, cat_cols)
    num_cols, cat_cols = removal_on_pvalue(targetcol, num_cols, cat_cols, data)
    num_cols, cat_cols = anovatyp1_removal(targetcol,num_cols,cat_cols,data)
    model, explanation, num_cols, cat_cols = finalmodel(targetcol,num_cols, cat_cols, data)
    fig, ax = plt.subplots(figsize=(10,5))
    sns.barplot(explanation.values,explanation.index)
    plt.show()
    print(model.summary())
    mse = model.mse_resid
    msr = model.mse_model
    print("R2 of the Final Model", model.rsquared_adj)
    n_iterations = 1000
    lower, upper = bootstrap_confidence(n_iterations, targetcol, num_cols, cat_cols, data)
    conf_interval, pred_interval = ci_pi(targetcol, num_cols, cat_cols,data)
    print("Confidence Interval for a given sample is:",conf_interval)
    print("prediction Interval for given sample is:",pred_interval)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 72 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   symboling                201 non-null    int64  
 1   wheel_base               201 non-null    float64
 2   curb_weight              201 non-null    int64  
 3   engine_size              201 non-null    int64  
 4   bore                     201 non-null    float64
 5   stroke                   201 non-null    float64
 6   compression_ratio        201 non-null    float64
 7   horsepower               201 non-null    float64
 8   peak_rpm                 201 non-null    float64
 9   city_mpg                 201 non-null    int64  
 10  highway_mpg              201 non-null    int64  
 11  price                    201 non-null    float64
 12  make_alfa_romero         201 non-null    int64  
 13  make_audi                201 non-null    int64  
 14  make_bmw                 2

In [ ]:
data['price']